<a href="https://colab.research.google.com/github/Shirsendu1260/ai-notebooks/blob/main/fake_news_detection_using_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [3]:
df_fake = pd.read_csv('/content/Fake.csv')
df_true = pd.read_csv('/content/True.csv')

In [4]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
df_fake.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [7]:
df_true.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [8]:
# Add 'class' columns on both dataframes (Fake and True represents 0 and 1 respectively)
df_fake['class'] = 0
df_true['class'] = 1

In [9]:
df_fake.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [10]:
df_true.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [11]:
df_fake.shape

(23481, 5)

In [12]:
df_true.shape

(21417, 5)

In [13]:
# Collect last 10 rows from actual 'Fake' dataframe for manual testing
df_fake_mt = df_fake.tail(10)

# Remove last 10 rows for manual testing from actual 'Fake' dataframe
for i in range(23480, 23470, -1):
  df_fake.drop([i], axis=0, inplace=True)

In [14]:
# Collect last 10 rows from actual 'True' dataframe for manual testing
df_true_mt = df_true.tail(10)

# Remove last 10 rows for manual testing from actual 'True' dataframe
for i in range(21416, 21406, -1):
  df_true.drop([i], axis=0, inplace=True)

In [15]:
df_fake_mt

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [16]:
df_true_mt

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [17]:
df_manual_testing = pd.concat([df_fake_mt, df_true_mt])
df_manual_testing

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [18]:
df_merge = pd.concat([df_fake, df_true])
df_merge

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
21402,Exclusive: Trump's Afghan decision may increas...,ON BOARD A U.S. MILITARY AIRCRAFT (Reuters) - ...,worldnews,"August 22, 2017",1
21403,U.S. puts more pressure on Pakistan to help wi...,WASHINGTON (Reuters) - The United States sugge...,worldnews,"August 21, 2017",1
21404,Exclusive: U.S. to withhold up to $290 million...,WASHINGTON (Reuters) - The United States has d...,worldnews,"August 22, 2017",1
21405,Trump talks tough on Pakistan's 'terrorist' ha...,ISLAMABAD (Reuters) - Outlining a new strategy...,worldnews,"August 22, 2017",1


In [19]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [20]:
df_merge1 = df_merge.drop(['title', 'subject', 'date'], axis=1)
df_merge1.isnull().sum()

text     0
class    0
dtype: int64

In [22]:
# Randomely arrange data within the dataframe
df_merge1 = df_merge1.sample(frac=1)
df_merge1

,text,class
9078,WASHINGTON (Reuters) - Democrats pushing for g...,1
12924,The Drudge Report has gained access to the rul...,0
9933,Fox News received an email from General John K...,0
10607,NEW YORK (Reuters) - The use of the #NeverTrum...,1
20802,If a black student was assaulted by a white un...,0
...,...,...
6161,"On Thursday, the Inspector General of the Stat...",0
2023,The following statements were posted to the ve...,1
4647,Donald Trump is so deplorable that even Cthulh...,0
19657,Thousands of students from Berkeley High Scho...,0


In [23]:
df_merge1.reset_index(inplace=True, drop=True)
df_merge1

,text,class
0,WASHINGTON (Reuters) - Democrats pushing for g...,1
1,The Drudge Report has gained access to the rul...,0
2,Fox News received an email from General John K...,0
3,NEW YORK (Reuters) - The use of the #NeverTrum...,1
4,If a black student was assaulted by a white un...,0
...,...,...
44873,"On Thursday, the Inspector General of the Stat...",0
44874,The following statements were posted to the ve...,1
44875,Donald Trump is so deplorable that even Cthulh...,0
44876,Thousands of students from Berkeley High Scho...,0


In [24]:
# Function to convert punctuations, special characters, symbols etc. to empty strings
def wordopt(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub('\\W', ' ', text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  return text

In [25]:
# Apply the above function to each row's 'text' column
df_merge1['text'] = df_merge1['text'].apply(wordopt)
df_merge1.head()

,text,class
0,washington reuters democrats pushing for g...,1
1,the drudge report has gained access to the rul...,0
2,fox news received an email from general john k...,0
3,new york reuters the use of the nevertrum...,1
4,if a black student was assaulted by a white un...,0


In [26]:
y = df_merge1['class']
x = df_merge1['text']

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.25)

In [33]:
x_train

34821     century wire says the field is now clear for ...
32094    late night host seth myers had a fun segment r...
37176    this one has to be seen to be believed liberal...
42547    there s a petition on change org going around ...
20873    when a state has legislated themselves into an...
                               ...                        
11284    big mouth billionaire and leftist author of th...
44732    andrew breitbart got under the skin of the lef...
38158    washington  reuters    the u s  supreme court ...
860                  https   www youtube com watch v ism  
15795    by now  everyone is aware of how donald trump ...
Name: text, Length: 33658, dtype: object

In [34]:
x_test

2915     conservative columnist ann coulter took her vo...
39470    dubai  reuters    state owned saudi arabian ai...
16174    question  how do you see the future of russian...
18424    millennial propaganda tool for the democrat pa...
38691     reuters    a reporter for the conservative we...
                               ...                        
4717     abu dhabi dubai  reuters    the united arab em...
18109    well  folks  your health care is safe for the ...
18388    washington  reuters    the white house said on...
37877    brussels  reuters    the eu executive launched...
32725    by now you ve probably heard about the remarks...
Name: text, Length: 11220, dtype: object

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

vt = TfidfVectorizer()
xv_train = vt.fit_transform(x_train)
xv_test = vt.transform(x_test)

In [41]:
xv_train

<33658x95186 sparse matrix of type '<class 'numpy.float64'>'
	with 6906494 stored elements in Compressed Sparse Row format>

In [42]:
xv_test

<11220x95186 sparse matrix of type '<class 'numpy.float64'>'
	with 2278882 stored elements in Compressed Sparse Row format>

In [43]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [44]:
model.fit(xv_train, y_train)

LogisticRegression()

In [49]:
y_pred = model.predict(xv_test)
y_pred

array([0, 1, 0, ..., 1, 1, 0])

In [50]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9868092691622103

In [51]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[5741,   83],
       [  65, 5331]])

In [52]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5824
           1       0.98      0.99      0.99      5396

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [56]:
def print_pred_msg(n):
  if n == 0:
    return 'Fake News'
  elif n == 1:
    return 'Not a Fake News'

def manual_testing(news):
  news_dictionary = {'text': [news]}
  df = pd.DataFrame(news_dictionary)
  df['text'] = df['text'].apply(wordopt)
  x_mt = df['text']
  xv_mt = vt.transform(x_mt)
  y_mt = model.predict(xv_mt)
  print('\n\nPrediction: {}' . format(print_pred_msg(y_mt[0])))

In [60]:
news = str(input('Enter a news: '))
manual_testing(news)

Enter a news: NEW DELHI (Reuters) - Indian police have arrested two Hindu farmers in connection with the killing of a Muslim man suspected of smuggling cows to a slaughter house, the investigating officer said on Wednesday. Hardline Hindu groups have gained prominence since Prime Minister Narendra Modi s nationalist Bharatiya Janata Party came to power in 2014, and the protection of cows, considered holy by many in the Hindu-majority country, is a priority for them. Police found the body of Ummar Khan, 35 on Friday on railway tracks in Alwar district, in the northwestern state of Rajasthan. The two suspects were detained on Monday and arrested the next day, said Rahul Prakash, the top police officer in Alwar.  It is a clear case of murder over the issue of cow protection and we have also found that the victim was running a cow-smuggling racket,  Prakash said. Khan and two accomplices were using a stolen vehicle to transport five cows when they were attacked by six members of a self-sty

In [61]:
news = str(input('Enter a news: '))
manual_testing(news)

Enter a news: The electronic war has not yet begun,    The JV teamThe Islamic State s hacking division has published personal details, including phone numbers, of the heads of the CIA, FBI and the National Security Agency as part of a coordinated hacking attack, according to reports by The Mail on Sunday. We are back  the hacking group declared.  We need years to publish what we have. We will raise our flag in the heart of Europe. #ISIS Cyber Caliphate Hacks 54k Twitter accounts, including ones of CIA and FBI officialshttps://t.co/2OCSWiqr3O pic.twitter.com/Ssl6jPoIQs  TRACterrorism.org (@TRACterrorism) November 9, 2015The group, calling itself  cyber-Caliphate  also posted details of 54,000 Twitter accounts, tweeting a link to the database, posted online.Once the passwords were put out online, the group encouraged ISIS supporters to take over the accounts and use them to put out ISIS propaganda. Due to the volume of information published, some victims were unaware they had been hacked